In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureSet
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent.parent

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [3]:
# create a GIS object instance; if you did not enter any information here, it defaults to anonymous access to ArcGIS Online
gis_portal = GIS(
    url=os.getenv('ESRI_PORTAL_URL'), 
    username=os.getenv('ESRI_PORTAL_USERNAME'),
    password=None if len(os.getenv('ESRI_PORTAL_PASSWORD')) is 0 else os.getenv('ESRI_PORTAL_PASSWORD')
)

gis_agol = GIS(
    url=os.getenv('ESRI_GIS_URL'), 
    username=os.getenv('ESRI_GIS_USERNAME'),
    password=None if len(os.getenv('ESRI_GIS_PASSWORD')) is 0 else os.getenv('ESRI_GIS_PASSWORD')
)

In [4]:
search_string = 'ace hardware'
code_naics = '44413005'
exclude_headquarters = True

In [235]:
from typing import Union
from warnings import warn

from arcgis.geometry import SpatialReference
from arcgis.gis import GIS
from modeling import Country, get_countries

def validate_spatial_reference_gis(spatial_reference:Union[str, int, dict, SpatialReference]):
    if isinstance(spatial_reference, dict):
        assert 'wkid' in spatial_reference.keys(), 'If providing spatial reference as a dictionary, it must conform '\
                                                   'look like {"wkid": <WKID>}, such as {"wkid": 4326}.'
        sr = spatial_reference
    elif isinstance(spatial_reference, str):
        assert spatial_reference.isdecimal(), 'If providing a string to identify a spatial reference, it must be a ' \
                                              'WKID for your desired spatial reference.'
        sr = {'wkid': spatial_reference}
    elif isinstance(spatial_reference, int):
        sr = {'wkid': spatial_reference}
    elif isinstance(spatial_reference, SpatialReference):
        sr = SpatialReference
    else:
        raise Exception('The spatial reference must be either a string or integer specify the WKID, a dictionary '\
                        'specifying the WKID such as {"wkid": 4326}, or a SpatialReference object.')
    return sr

def get_businesses_gis(area_of_interest:pd.DataFrame, gis:GIS, search_string:str=None, code_naics:Union[str, list]=None,
                       code_sic:Union[str, list]=None, exclude_headquarters:bool=True, country:Country=None, 
                       output_spatial_reference:Union[str, int, dict, SpatialReference]={'wkid': 4326}) -> pd.DataFrame:

    # list of fields to lighten the output payload
    out_flds = ['LOCNUM', 'CONAME', 'NAICSDESC', 'NAICS', 'SIC', 'SOURCE', 'PUBPRV','FRNCOD', 'ISCODE', 'CITY', 'ZIP', 
                'STATE', 'HDBRCHDESC']
    
    # begin to build up the request parameter payload
    params = {
        'f': 'json',
        'returngeometry': True,
        'outsr': validate_spatial_reference_gis(spatial_reference),
        'fields': out_flds
    }
    
    # make sure a country is somehow explicitly specified
    if '_cntry' in area_of_interest.attrs:
        params['sourcecountry'] = area_of_interest.attrs['_cntry'].iso2
        
    # if a country object was explicitly passed in - easy peasy lemon squeezy
    elif country is not None and isinstance(country, str):
        params['sourcecountry'] = country.iso2
    
    # if there is not a country to work with, bingo out
    else:
        raise Exception('Either the input dataframe must have been created using the modeling ' \
                        'module to retrieve standard geographies, or a country ojbect must be ' \
                        'explicitly specified in the input parameters.')
    
    # make sure some sort of filter is being applied
    assert (search_string is not None) or (code_naics is not None), 'You must provide either a search string or ' \
                                                                    'NAICS code or list of codes to search for ' \
                                                                    'businesses.' 

    # populate the rest of the search parameters
    params['searchstring'] = search_string
    params['businesstypefilters'] = [
        {'Classification': 'NAICS','Codes':[code_naics]},
        {'Classification': 'SIC', 'Codes': [code_sic]}
    ]
    
    # if the input Spatially Enabled DataFrame was created using the modeling module to get standard geographies
    if 'parent_geo' in area_of_interest.attrs.keys():
        params['spatialfilter'] = {
            "Boundaries": {
                "StdLayer": {
                    "ID":area_of_interest.attrs['parent_geo']['resource'],
                    "GeographyIDs":area_of_interest.attrs['parent_geo']['id']
                }
            }
        }
        
    # if just a normal spatially enabled dataframe, we can still use the geometry
    else:
        params['spatialfilter'] = {
            "Boundaries": {
                "recordSet": df.spatial.to_featureset().to_dict()
            }
        }

    # retrieve the businesses from the REST endpoint
    url = f'{gis.properties.helperServices.geoenrichment.url}/SelectBusinesses'
    r_json = gis._con.post(url, params=params)
    
    # ensure a valid result is received
    if 'error' in r_json.keys():
        err = r_json['error']
        raise Exception(f'Error in searching using Business Analyst SelectBusinesses REST endpoint. Error Code '
                        f'{err["code"]}: {err["message"]}')
        
    else:
        
        # plow through all the messages to see if there are any errors
        err_msg_lst =[]
        for val in r_json['messages']:
            if 'description' in val.keys():
                if 'error' in val['description'].lower():
                    err_msg_lst.append(val)
        
        # if an error message is found
        if len(err_msg_lst):
            err = err_msg_lst[0]
            raise Exception(f'Server error encoutered in searching using Business Analyst SelectBusinesses REST endpoint. '
                            f'Error ID: {err["id"]}, Type: {err["type"]}, Description: {err["description"]}')
        
    # extract the feature list out of the json response
    feature_lst = r_json['results'][0]['value']['features']
    
    # make sure something was found
    if len(feature_lst) == 0:
        warn('Although the request was valid and no errors were encountered, no businesses were found.')
    
    # convert the features to a Spatially Enabled Pandas DataFrame
    res_df = FeatureSet(feature_lst).sdf
    
    # reorganize the schema a little
    cols = [c for c in out_flds if c in res_df.columns] + ['SHAPE']
    res_df = res_df[cols]
    
    # if not wanting to keep headquarters, normally the case for forecasting modeling, filter them out
    if exclude_headquarters:
        res_df = res_df[~res_df['HDBRCHDESC'].str.lower().str.match('headquarters')].reset_index(drop=True)
        
    # drop the headquarters or branch column since only used to filter if necessary
    res_df.drop(columns='HDBRCHDESC', inplace=True)
        
    return res_df

In [4]:
gis_portal = GIS(
    url=os.getenv('ESRI_PORTAL_URL'), 
    username=os.getenv('ESRI_PORTAL_USERNAME'),
    password=os.getenv('ESRI_PORTAL_PASSWORD')
)

usa = modeling.Country('USA', gis_portal)

usa

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [5]:
aoi_df = usa.cbsas.get('seattle', return_geometry=True)

aoi_df

,ID,NAME,SHAPE
0,42660,"Seattle-Tacoma-Bellevue, WA Metropolitan Stati...","{""rings"": [[[-121.68612849977673, 48.298988999..."


In [65]:
%%time
brnd_df = usa.business.get_by_name('ace hardware', aoi_df)
brnd_df['brand_name_category'] = 'ACE HARDWARE'

brnd_df.head()

CPU times: user 28.6 ms, sys: 3.56 ms, total: 32.2 ms
Wall time: 809 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category
0,174746248,MAGNOLIA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98199,WA,"{""x"": -122.39802, ""y"": 47.6402850000001, ""spat...",174746248,MAGNOLIA ACE HARDWARE,ACE HARDWARE
1,174841932,ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,LAKE STEVENS,98258,WA,"{""x"": -122.1072885, ""y"": 47.998143, ""spatialRe...",174841932,ACE HARDWARE,ACE HARDWARE
2,174852467,CARRS ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MARYSVILLE,98270,WA,"{""x"": -122.1761745, ""y"": 48.05064, ""spatialRef...",174852467,CARRS ACE HARDWARE,ACE HARDWARE
3,216082099,SOUTH END ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SPANAWAY,98387,WA,"{""x"": -122.4344385, ""y"": 47.087136, ""spatialRe...",216082099,SOUTH END ACE HARDWARE,ACE HARDWARE
4,251373601,CEDAR PLAZA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MOUNTLAKE TER,98043,WA,"{""x"": -122.2900785, ""y"": 47.7909450000001, ""sp...",251373601,CEDAR PLAZA ACE HARDWARE,ACE HARDWARE


In [66]:
%%time
comp_df = usa.business.get_competition(brnd_df, aoi_df, local_threshold=3)

comp_df

CPU times: user 27 ms, sys: 2.57 ms, total: 29.6 ms
Wall time: 249 ms


,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MC LENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,DUVALL TRUE VALUE HARDWARE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,GATEWAY TRUE VALUE HARDWARE,local_brand
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC


In [67]:
brand_str = r'true value|true vl'

brand_filter = comp_df.brand_name.str.lower().str.contains(brand_str, regex=True)

comp_df[brand_filter]

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,DUVALL TRUE VALUE HARDWARE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,GATEWAY TRUE VALUE HARDWARE,local_brand
6,174594002,JUNCTION TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98116,WA,"{""x"": -122.3886375, ""y"": 47.559537, ""spatialRe...",174594002,JUNCTION TRUE VALUE HARDWARE,local_brand
12,174888537,SNOHOMISH CO-OP TRUE VALUE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SNOHOMISH,98290,WA,"{""x"": -122.087522, ""y"": 47.9117034400001, ""spa...",174888537,SNOHOMISH CO-OP TRUE VALUE,local_brand
25,415082473,CLEARVIEW TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SNOHOMISH,98296,WA,"{""x"": -122.119038, ""y"": 47.8473480000001, ""spa...",415082473,CLEARVIEW TRUE VALUE HARDWARE,local_brand
39,452112691,GREENWOOD TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3556615, ""y"": 47.6809650000001, ""sp...",452112691,GREENWOOD TRUE VALUE HARDWARE,local_brand
42,543667539,CARMICHAEL'S TRUE VALUE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SNOQUALMIE,98065,WA,"{""x"": -121.8231, ""y"": 47.527434, ""spatialRefer...",543667539,CARMICHAEL'S TRUE VALUE,local_brand
64,803909134,TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,MERCER ISLAND,98040,WA,"{""x"": -122.2344225, ""y"": 47.586123, ""spatialRe...",803909134,TRUE VALUE HARDWARE,local_brand
68,804462166,BROOKDALE LUMBER TRUE VALUE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,TACOMA,98444,WA,"{""x"": -122.4350685, ""y"": 47.133135, ""spatialRe...",804462166,BROOKDALE LUMBER TRUE VALUE,local_brand
74,979005980,CITY PEOPLE SANDPOINT TRUE VL,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98105,WA,"{""x"": -122.2753095, ""y"": 47.668032, ""spatialRe...",979005980,CITY PEOPLE SANDPOINT TRUE VL,local_brand


In [68]:
comp_df.loc[brand_filter, 'brand_name'] = 'TRUE VALUE'

comp_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MC LENDON HARDWARE,local_brand
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,local_brand
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,local_brand
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC


In [74]:
from modeling.businesses import Business

Business.calculate_brand_name_category(comp_df, 3)

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category,brand_comp
0,002890986,MC LENDON HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SUMNER,98390,WA,"{""x"": -122.242365, ""y"": 47.2046040000001, ""spa...",002890986,MC LENDON HARDWARE,local_brand,competition
1,006128854,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,RENTON,98057,WA,"{""x"": -122.2140195, ""y"": 47.477943, ""spatialRe...",006128854,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC,competition
2,174245191,DUVALL TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,DUVALL,98019,WA,"{""x"": -121.9853835, ""y"": 47.738907, ""spatialRe...",174245191,TRUE VALUE,TRUE VALUE,competition
3,174262691,GATEWAY TRUE VALUE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,ENUMCLAW,98022,WA,"{""x"": -121.9876155, ""y"": 47.2019940000001, ""sp...",174262691,TRUE VALUE,TRUE VALUE,competition
4,174471722,TWEEDY & POPP HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,2,,SEATTLE,98103,WA,"{""x"": -122.3357134, ""y"": 47.6612959300001, ""sp...",174471722,TWEEDY & POPP HARDWARE,local_brand,competition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand,competition
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand,competition
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand,competition
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC,competition


In [70]:
pd.Series(comp_df.brand_name_category.unique()).sort_values().reset_index(drop=True)

0    MCLENDON HARDWARE INC
1               TRUE VALUE
2              local_brand
dtype: object

In [71]:
brnd_df['brand_comp'] = 'brand'
comp_df['brand_comp'] = 'competition'

In [72]:
loc_df = pd.concat([brnd_df, comp_df])
loc_df

,LOCNUM,CONAME,NAICSDESC,NAICS,SIC,SOURCE,PUBPRV,FRNCOD,ISCODE,CITY,ZIP,STATE,SHAPE,id,brand_name,brand_name_category,brand_comp
0,174746248,MAGNOLIA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SEATTLE,98199,WA,"{""x"": -122.39802, ""y"": 47.6402850000001, ""spat...",174746248,MAGNOLIA ACE HARDWARE,ACE HARDWARE,brand
1,174841932,ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,LAKE STEVENS,98258,WA,"{""x"": -122.1072885, ""y"": 47.998143, ""spatialRe...",174841932,ACE HARDWARE,ACE HARDWARE,brand
2,174852467,CARRS ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MARYSVILLE,98270,WA,"{""x"": -122.1761745, ""y"": 48.05064, ""spatialRef...",174852467,CARRS ACE HARDWARE,ACE HARDWARE,brand
3,216082099,SOUTH END ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,SPANAWAY,98387,WA,"{""x"": -122.4344385, ""y"": 47.087136, ""spatialRe...",216082099,SOUTH END ACE HARDWARE,ACE HARDWARE,brand
4,251373601,CEDAR PLAZA ACE HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,1,,MOUNTLAKE TER,98043,WA,"{""x"": -122.2900785, ""y"": 47.7909450000001, ""sp...",251373601,CEDAR PLAZA ACE HARDWARE,ACE HARDWARE,brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,933642043,AIRCRAFT HARDWARE,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,EVERETT,98204,WA,"{""x"": -122.2741203, ""y"": 47.90709365, ""spatial...",933642043,AIRCRAFT HARDWARE,local_brand,competition
71,950645150,JENSEN PLUMBING LLC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,SEATTLE,98146,WA,"{""x"": -122.35702, ""y"": 47.4991650000001, ""spat...",950645150,JENSEN PLUMBING LLC,local_brand,competition
72,966020547,EQUIPMENT SALES & SURPLUS,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,AUBURN,98001,WA,"{""x"": -122.2504785, ""y"": 47.325294, ""spatialRe...",966020547,EQUIPMENT SALES & SURPLUS,local_brand,competition
73,967153974,MCLENDON HARDWARE INC,HARDWARE-RETAIL,44413005,525104,INFOGROUP,,,,KENT,98031,WA,"{""x"": -122.200308, ""y"": 47.3884200000001, ""spa...",967153974,MCLENDON HARDWARE INC,MCLENDON HARDWARE INC,competition


In [73]:
loc_df.brand_name_category.value_counts().sort_values(ascending=False)

local_brand              60
ACE HARDWARE             37
TRUE VALUE               10
MCLENDON HARDWARE INC     5
Name: brand_name_category, dtype: int64